In [ ]:
import glob
import math
import uproot
import numpy as np
import pandas as pd
import pkgutil
import uproot_methods

import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.colors import LightSource
%matplotlib inline

import mplhep as hep
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

import max_functions as mf

plt.style.use(hep.style.ROOT) # For now ROOT defaults to CMS

path = r'D:\UniFRK\SEXTANT_sept2018\S-TFMoX\enant=S_hel=+1_KE=3.0eV' # use your path
all_files = glob.glob(path + "/*.dat")

li = []
colnames=["phi","cos(theta)","value"] 

# how to load multiple files http://jonathansoma.com/lede/foundations-2017/classes/working-with-many-files/class/
for filename in all_files:
    df = pd.read_csv(filename, delimiter=r"\s+", names=colnames, header=None) 
    # or delim_whitespace=True, it is faste
    # r"\s+" is a regex (regular expression)
    df["filename"]=filename.split("\\")[-1].split(".")[0] # adding a column with the file name
    df["filename"]=df["filename"].str.replace(" ","") # corrects for spaces
    df["filename"]=df["filename"].astype("category")
    li.append(df) # a unique DataFrame

frame = pd.concat(li, axis=0)
frame_srt=frame.groupby("filename")


In [ ]:
frame.loc[frame["filename"] == "1_1"] # first way of selecting the right file
frame_set = frame.set_index("filename") # build a multiindex using the categories of filename

In [ ]:
# Single 2D maps from pandas without tranforming into numpy
frame.loc[frame["filename"] == "1_1"].plot.hexbin(x="phi", y="cos(theta)", C="value", sharex=False, gridsize=50)

In [ ]:
# converting to numpy arrays to plot
phi = frame.loc[frame["filename"] == "1_1"].iloc[:,0].unique() # phi
ctheta = frame.loc[frame["filename"] == "1_1"].iloc[:,1].unique() # cos(theta)
counts = frame.loc[frame["filename"] == "1_1"].iloc[:,2].to_numpy() # values

print(phi.shape, ctheta.shape, counts.shape)
print(type(phi), type(ctheta), type(counts))
counts_matrix = counts.reshape(200,100) # reshape(rows=x,columns=y)

print("number of ROWS (x) = ", len(counts_matrix[:,1]), "number of COLUMNS (y for reshaping) = ", len(counts_matrix[1,:]))

In [ ]:
# plt.imshow(z_matrix)
plt.imshow(counts_matrix.T,extent=[phi[0],phi[-1],ctheta[0],ctheta[-1]], aspect=180)
ax = plt.axes()
ax.set_xlabel('phi [DEG]')
ax.set_ylabel('cos(theta) [adm]')
plt.colorbar()

In [ ]:
# countour plot
cmap = mpl.cm.viridis
cmap.set_under('w')


fig1, ax = plt.subplots(constrained_layout=True)

CS = ax.contourf(phi,ctheta,counts_matrix.T, levels=20, extent=[phi[0],phi[-1],ctheta[0],ctheta[-1]], vmin=0.0001)
ax.set_xlabel('phi [DEG]')
ax.set_ylabel('cos(theta) [adm]')
fig1.colorbar(CS)

# OLD WAY OF PROCEDING!! Note unique values and extent, all the 72 plots
phi = frame.loc[frame["filename"] == "1_1"].iloc[:,0].unique() # phi
ctheta = frame.loc[frame["filename"] == "1_1"].iloc[:,1].unique() # cos(theta)

fig = plt.figure(figsize=(50, 24),dpi=80) # figsize in inche by defauls: 1 in = 2.8 cm
for filename,group,i in zip(frame_srt,frame_srt,range(1,73)) :
    counts_str = group[1]["value"].to_numpy()
    counts_matrix = counts_str.reshape(200,100)
    ax = fig.add_subplot(6, 12, i)
    ax.contourf(phi,ctheta,counts_matrix, levels=20, extent=[phi[0],phi[-1],ctheta[0],ctheta[-1]], vmin=0.0001)
    ax.text(i, 1+i//12, i, ha="center", va="center", color="red")

fig.tight_layout()
# plt.show()

In [ ]:
# all the 72 plots
# the theoretical MFPAD has the form 1_1 ... 1_12, 2_1 ... 2_12 therefore it is ok the (6,12) subplot
# 6 levels of cos(theta) (y), 12 levels of phi (x)
phi = frame.loc[frame["filename"] == "1_1"].iloc[:,0].to_numpy() # phi
ctheta = frame.loc[frame["filename"] == "1_1"].iloc[:,1].to_numpy() # cos(theta)

fig = plt.figure(figsize=(50, 24),dpi=80) # figsize in inche by defauls: 1 in = 2.8 cm
for filename,group,i in zip(frame_srt,frame_srt,range(1,73)) :
    counts_str = group[1]["value"].to_numpy()
    ax = fig.add_subplot(6, 12, i)
    ax.contourf(phi.reshape(200,100),ctheta.reshape(200,100),counts_str.reshape(200,100), levels=20, vmin=0.0001)
    ax.text(i, 1+i//12, i, ha="center", va="center", color="red")

fig.tight_layout()
# plt.show()

# alterative methods for subplots: not working well, lets marks

fig, axs = plt.subplots(2,5, figsize=(50, 24), sharex=True, sharey=True)
axs = axs.ravel()

# all the 72 plots
phi = frame.loc[frame["filename"] == "1_1"].iloc[:,0].unique() # phi
ctetha = frame.loc[frame["filename"] == "1_1"].iloc[:,1].unique() # cos(theta)

for filename,group,i in zip(frame_srt,frame_srt,range(1,73)) :
    counts_str = group[1]["value"].to_numpy()
    counts_matrix = counts_str.reshape(200,100)
    ax = fig.add_subplot(6, 12, i)
    ax.contourf(phi,ctheta,counts_matrix.T, levels=20, extent=[phi[0],phi[-1],ctheta[0],ctheta[-1]], vmin=0.0001)

fig.tight_layout()
plt.show()

In [ ]:
# all the 72 plots: no borders & sharing axis


phi = frame.loc[frame["filename"] == "1_1"].iloc[:,0].to_numpy() # phi
ctheta = frame.loc[frame["filename"] == "1_1"].iloc[:,1].to_numpy() # cos(theta)

fig, axes = plt.subplots(6,12, figsize=(50, 24), sharex='col', sharey='row')
fig.subplots_adjust(hspace = .5, wspace=.5)

for filename,group,ax in zip(frame_srt,frame_srt,axes.flatten()) :
    counts_str = group[1]["value"].to_numpy()
    counts_matrix = counts_str.reshape(200,100)
    ax.contourf(phi.reshape(200,100),ctheta.reshape(200,100),counts_str.reshape(200,100), levels=20, vmin=0.0001)
    
fig.tight_layout()
plt.show()

In [ ]:
# To convert in sphrerical coordinates!!

phi = frame.loc[frame["filename"] == "1_1"].iloc[:,0].to_numpy() # phi
ctheta = frame.loc[frame["filename"] == "1_1"].iloc[:,1].to_numpy() # cos(theta)
counts_str = frame.loc[frame["filename"] == "1_1"].iloc[:,2].to_numpy() # counts

theta = np.arccos(ctheta)
phi_rad = phi * 3.14159265359/180.

# convertion to spherical coordinates 1D vectors of shape (2000,1)
x = counts_str * np.sin(theta) * np.cos(phi_rad)
y = counts_str * np.sin(theta) * np.sin(phi_rad)
z_str = counts_str * ctheta

# distance from the origin for the cmap
# remember do not use math, but np for fast operations on vectors
d_str = np.sqrt(x**2+y**2+z_str**2)

X,Y = np.meshgrid(x, y, sparse=True)

fig = plt.figure()
fig, ax = plt.subplots(1,2,subplot_kw={"projection": "3d"})
# alternative sintax
# fig = plt.figure()
# ax = fig.gca(projection='3d')

ax = fig.add_subplot(1, 2, 1, projection='3d')
ax.scatter(x, y, z_str, c=d_str, cmap='viridis', alpha=0.75) # X,Y,Z 1D vectors -> everything ok

# Z should be 2D
# ax = fig.add_subplot(1, 3, 2, projection='3d')
# ax.plot_wireframe(X,Y,Z)

ax = fig.add_subplot(1, 2, 2, projection='3d')
ax.plot_trisurf(x, y, z_str, antialiased=True)

fig.tight_layout()
plt.show()

In [ ]:

phi = frame.loc[frame["filename"] == "1_1"].iloc[:,0].to_numpy() # phi
ctheta = frame.loc[frame["filename"] == "1_1"].iloc[:,1].to_numpy() # cos(theta)
counts_str = frame.loc[frame["filename"] == "1_1"].iloc[:,2].to_numpy() # counts

theta = np.arccos(ctheta)
phi_rad = phi * 3.14159265359/180.

# convertion to spherical coordinates 1D vectors of shape (2000,1)
x = counts_str * np.sin(theta) * np.cos(phi_rad)
y = counts_str * np.sin(theta) * np.sin(phi_rad)
z_str = counts_str * ctheta

# subsampling with slicing to reduce the computational effort
X,Y = np.meshgrid(x[0:-1:3], y[0:-1:3], sparse=True)
Z_1, Z_2 = np.meshgrid(x[0:-1:3], y[0:-1:3], sparse=True)

fig = plt.figure()
fig, ax = plt.subplots(1,2, subplot_kw={"projection": "3d"}, figsize=(20,10))

ax = fig.add_subplot(1, 2, 1, projection='3d')
ax.plot_surface(X,Y,Z_1)

ax = fig.add_subplot(1, 2, 2, projection='3d')
ax.plot_surface(X,Y,Z_2)

plt.show()

In [ ]:
# To convert in sphrerical coordinates!!
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
from matplotlib.colors import LightSource

phi = frame.loc[frame["filename"] == "1_1"].iloc[:,0].to_numpy() # phi
ctheta = frame.loc[frame["filename"] == "1_1"].iloc[:,1].to_numpy() # cos(theta)
counts_str = frame.loc[frame["filename"] == "1_1"].iloc[:,2].to_numpy() # counts

theta_rad = np.arccos(ctheta)
phi_rad = phi*np.pi/180.

# convertion to cartesian coordinates 1D vectors of shape (20000,1)
x = counts_str * np.sin(theta_rad) * np.cos(phi_rad)
y = counts_str * np.sin(theta_rad) * np.sin(phi_rad)
z_str = counts_str * ctheta

verts = [list(zip(x, y, z_str))]

fig = plt.figure()
ax = Axes3D(fig)

# facecolor='g'
ax.add_collection3d(Poly3DCollection(verts, cmap="hot", alpha=0.9))
ls = LightSource(azdeg=225.0, altdeg=45.0)
# ax.auto_scale_xyz(verts[:,0], verts[:,1], verts[:,2])
ax.set_xlim3d(x.min(), x.max())
ax.set_ylim3d(y.min(), y.max())
ax.set_zlim3d(z_str.min(), z_str.max())

plt.show()